In [25]:
# import standard stuff...
import os
import sys
import cellml

First we want to parse the CellML document that we are going to annotate. We parse in non-strict mode to allow any version CellML model to be used.

In [26]:
model_file = 'cellml/delay_test.cellml'
model = cellml.parse_model(model_file, False)

The method "parse_model" found 1 issues:
    - Given model is a CellML 1.1 model, the parser will try to represent this model in CellML 2.0.


Now we create an annotator object that we can use to manage the metadata IDs on our model

In [27]:
from libcellml import Annotator
annotator = Annotator()
annotator.setModel(model)

Make sure all entities in the model have an ID and make sure that all IDs in the model are unique.

In [28]:
if annotator.assignAllIds():
    print('Some entities have been assigned an ID, you should save the model!')
else:
    print('Everything already had an ID.')

duplicates = annotator.duplicateIds()
if len(duplicates) > 0:
    print("There are some duplicate IDs, behaviour may be unreliable...")
    print(duplicates)

Some entities have been assigned an ID, you should save the model!


Since this test model has some messed up IDs, lets just blow them away and start again...

In [29]:
annotator.clearAllIds()
annotator.assignAllIds()
model_string = cellml.print_model(model)
print(model_string)

# and save the updated model to a new file - note, we need the model filename for making our annotations later
model_file = 'cellml/delay_test-updated-ids.cellml'
with open(model_file, 'w') as f:
    f.write(model_string)

<?xml version="1.0" encoding="UTF-8"?>
<model xmlns="http://www.cellml.org/cellml/2.0#" name="my_model" id="b4da60">
  <units name="per_s" id="b4da61">
    <unit exponent="-1" units="second" id="b4da62"/>
  </units>
  <component name="delay_test" id="b4da63">
    <variable name="t" units="second" id="b4da64"/>
    <variable name="f_a" units="per_s" id="b4da65"/>
    <variable name="V_delay" units="per_s" id="b4da66"/>
    <variable name="V" units="dimensionless" initial_value="0.0" id="b4da67"/>
    <variable name="out" units="dimensionless" id="b4da68"/>
    <variable name="delay" units="second" initial_value="0.5" id="b4da69"/>
    <variable name="k_e" units="dimensionless" initial_value="1.0" id="b4da6a"/>
    <math xmlns="http://www.w3.org/1998/Math/MathML" xmlns:cellml="http://www.cellml.org/cellml/2.0#">
      <apply>
        <eq/>
        <apply>
          <diff/>
          <bvar>
            <ci>t</ci>
          </bvar>
          <ci>V</ci>
        </apply>
        <apply>
    

Now we can do some annotation using pyOmexMeta (see https://github.com/sys-bio/pyomexmeta-binder-notebook) - except it doesn't work for CellML 2.0 models. So for now we work directly with RDF.

In [30]:
# get the ID of the variables we want to annotate
variable_to_delay = model.component('delay_test').variable('V').id()
delay_variable = model.component('delay_test').variable('V_delay').id()
delay_amount = model.component('delay_test').variable('delay').id()
independent_variable = model.component('delay_test').variable('t').id()

output_variables = [model.component('delay_test').variable('t').id(), 
                    model.component('delay_test').variable('V_delay').id(), 
                    model.component('delay_test').variable('V').id(), 
                    model.component('delay_test').variable('out').id(), 
                    model.component('delay_test').variable('f_a').id()]

# make an RDF Graph to add annotations to - using rdflib
from rdflib import Graph, Literal, RDF, URIRef
from rdflib.namespace import DCTERMS

g = Graph()

# define some URIs for things we need

# use this URI to identify delayed variables - not the perfect URI, but will do for now
#     This is actually "delayed differential equation model" from the MAMO ontology
#delay_variable_uri = URIRef('http://identifiers.org/mamo/MAMO_0000089')
delay_variable_uri = URIRef('https://github.com/nickerso/libcellml-python-utils/properties.rst#delay-variable')
variable_to_delay_uri = URIRef('https://github.com/nickerso/libcellml-python-utils/properties.rst#variable-to-delay')
independent_variable_uri = URIRef('https://github.com/nickerso/libcellml-python-utils/properties.rst#variable-to-delay')
delay_amount_uri = URIRef('https://github.com/nickerso/libcellml-python-utils/properties.rst#delay-amount')

# use this for some random thing that I want to define - http://example.com is a good base for things that will never resolve
stuff_uri = URIRef('http://example.com/cool-thing#21')

# a "readout" variable that we maybe want to connect to something external?
timecourse_readout_uri = URIRef('http://identifiers.org/mamo/MAMO_0000031')
output_variable_uri = URIRef('http://identifiers.org/mamo/MAMO_0000018')
input_variable_uri = URIRef('http://identifiers.org/mamo/MAMO_0000017')

# Create an RDF URI node for our variable to use as the subject for multiple triples
# note: we are going to serialise the RDF graph into the same folder, so we need a URI that is relative to the intended file
variable_to_delay_name_uri = URIRef('delay_test-updated-ids.cellml' + '#' + variable_to_delay)
delay_variable_name_uri = URIRef('delay_test-updated-ids.cellml' + '#' + delay_variable)
independent_variable_name_uri = URIRef('delay_test-updated-ids.cellml' + '#' + independent_variable)
delay_amount_name_uri = URIRef('delay_test-updated-ids.cellml' + '#' + delay_amount)

# Add triples using store's add() method.
# We're using the Dublin Core term "type" to associate the variable with the delay...
g.add((variable_to_delay_name_uri, DCTERMS.type, variable_to_delay_uri))
g.add((variable_to_delay_name_uri, delay_variable_uri, delay_variable_name_uri))
g.add((variable_to_delay_name_uri, independent_variable_uri, independent_variable_name_uri))
g.add((variable_to_delay_name_uri, delay_amount_uri, delay_amount_name_uri))
# Set timecourse readout variables
for ov in output_variables:
    output_uri = URIRef('delay_test-updated-ids.cellml' + '#' + ov)
    g.add((output_uri, DCTERMS.type, timecourse_readout_uri))

# print all the data in the turtle format
print(g.serialize(format='ttl'))

# and save to a file
with open('cellml/delay_test-updated-ids--annotations.ttl', 'w') as f:
    f.write(g.serialize(format='ttl'))

@prefix ns1: <https://github.com/nickerso/libcellml-python-utils/properties.rst#> .
@prefix ns2: <http://purl.org/dc/terms/> .

<delay_test-updated-ids.cellml#b4da65> ns2:type <http://identifiers.org/mamo/MAMO_0000031> .

<delay_test-updated-ids.cellml#b4da67> ns2:type <http://identifiers.org/mamo/MAMO_0000031>,
        ns1:variable-to-delay ;
    ns1:delay-amount <delay_test-updated-ids.cellml#b4da69> ;
    ns1:delay-variable <delay_test-updated-ids.cellml#b4da66> ;
    ns1:variable-to-delay <delay_test-updated-ids.cellml#b4da64> .

<delay_test-updated-ids.cellml#b4da68> ns2:type <http://identifiers.org/mamo/MAMO_0000031> .

<delay_test-updated-ids.cellml#b4da64> ns2:type <http://identifiers.org/mamo/MAMO_0000031> .

<delay_test-updated-ids.cellml#b4da66> ns2:type <http://identifiers.org/mamo/MAMO_0000031> .


